In [1]:
#%%capture
#!wget "https://repo1.maven.org/maven2/org/apache/spark/spark-sql-kafka-0-10_2.12/3.5.0/spark-sql-kafka-0-10_2.12-3.5.0.jar"
#!wget "https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-10_2.12/3.5.0/spark-streaming-kafka-0-10_2.12-3.5.0.jar"

In [2]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.5.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0 pyspark-shell'

In [3]:
import pandas as pd
import os
import pickle
import json
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder.appName("OCR").getOrCreate()
spark.sparkContext.setLogLevel("WARN")

In [4]:
!pip install mistralai

In [5]:
#Need to restart kernel once after finishing install mistral

In [6]:
### Create mistral key from https://colab.research.google.com/github/mistralai/cookbook/blob/main/mistral/ocr/structured_ocr.ipynb#scrollTo=NhwM0aITt7ti

In [7]:
# 1. Read Stream from Kafka
raw_stream = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "10.10.83.206:8097") \
  .option("subscribe", "input") \
  .option("startingOffsets", "latest") \
  .load()

raw_stream.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [8]:
# For debugging
if False:
    query = raw_stream \
      .selectExpr("CAST(key AS STRING)") \
      .writeStream \
      .outputMode("update") \
      .format("console") \
      .option("truncate", False) \
      .start()
    
    query.awaitTermination()

In [9]:
processed_data = raw_stream.select(
    col("value").cast("BINARY").alias("png_data"),
    col("key").cast(StringType()).alias("filename")
)

In [10]:
#import requests
from pyspark.sql import Row
from mistralai import DocumentURLChunk, ImageURLChunk, TextChunk
import base64
import time
from mistralai import Mistral
# Initialize Mistral client with API key
api_key = ''
client = Mistral(api_key=api_key)

# ---------------------
def call_mistral_ocr(row: Row):

    filename = row['filename']
    png_bytes = row['png_data']
    
    # Encode image as base64 for API
    encoded = base64.b64encode(png_bytes).decode()
    base64_data_url = f"data:image/jpeg;base64,{encoded}"
    
    # Process image with OCR
    image_response = client.ocr.process(
        document=ImageURLChunk(image_url=base64_data_url),
        model="mistral-ocr-latest"
    )

    image_ocr_markdown = image_response.pages[0].markdown

    # Get structured response from model
    chat_response = client.chat.complete(
        model="pixtral-12b-latest",
        messages=[
            {
                "role": "user",
                "content": [
                    ImageURLChunk(image_url=base64_data_url),
                    TextChunk(
                        text=(
                            f"This is image's OCR in markdown:\n\n{image_ocr_markdown}\n.\n"
                            "Convert this into a sensible structured json response. "
                            "The output should be strictly be json with no extra commentary"
                        )
                    ),
                ],
            }
        ],
        response_format={"type": "json_object"},
        temperature=0,
    )

    # Parse and return JSON response
    response_dict = json.loads(chat_response.choices[0].message.content)
    json_string = json.dumps(response_dict, indent=4)
    return filename, response_dict

def process_batch(df, batch_id):

    start_time_full = time.time()
    """
    Function executed for every micro-batch of the Structured Stream.
    Collects the rows and processes them with the OCR function.
    """
    # ⚠️ WARNING: .collect() brings ALL data to the driver program. 
    # Use for testing/low-volume only. For high-volume, consider a custom Kafka Connect Sink.
    rows = df.collect()
    
    results = []
    for row in rows:
        start_time_ai = time.time()
        filename, ocr_text = call_mistral_ocr(row)
        end_time_ai = time.time()
        total_ai_duration = end_time_ai - start_time_ai
        print(f"   > AI Call Duration: {total_ai_duration:.4f} seconds")
        
        if filename is None:
            filename = f"image_{batch_id}"
        results.append((filename, ocr_text))
        print(f"OCR Result for {filename}: {ocr_text}") # Log first 80 chars

    schema = StructType([
        StructField("filename", StringType(), True),
        StructField("ocr_text", StringType(), True) 
    ])

    if results:
        results_df = spark.createDataFrame(results, schema)

        # 3. Write results to JSON files (The Solution)
        # Use the batch_id to create a unique subdirectory for the JSON files
        batch_output_path = os.path.join("/home/jovyan/json", f"batch_{batch_id}")

        results_df.write \
            .format("json") \
            .mode("overwrite") \
            .save(batch_output_path)
        
        print(f"Successfully wrote OCR results for Batch {batch_id} to: {batch_output_path}")
        end_time_full = time.time()
        total_duration = end_time_full - start_time_full
        print(f"   > Total Duration: {total_duration:.4f} seconds")
        time_usage = total_duration - total_ai_duration
        print(f"   > Total Function Duration (exclude AI time): {time_usage:.4f} seconds")
    

In [11]:
query = processed_data.writeStream \
    .foreachBatch(process_batch) \
    .start() 

query.awaitTermination()

   > AI Call Duration: 7.4165 seconds
OCR Result for image_1: {'summary': {'course': 'พลเมืองกับการลงมือแก้ปัญหา', 'semester': 'ภาคการศึกษาที่ 2/2564', 'student_info': {'name': 'aycsphdph', 'year': '2562', 'registration_number': '6418490543', 'group': 'F5'}, 'assessment_time': '15 นาที'}, 'questions': [{'question': '1. คุณสมบัติพลเมืองที่นักศึกษาได้เรียนรู้ จนถึงคาบที่ 8 นี้ มีอะไรบ้าง?'}, {'question': '2. 1) พัฒนาที่ทำการสอนและการสอนของตนเอง'}, {'question': '2. 2) พัฒนาเรียนรู้และการสอนของตนเองในการสอนของตนเอง'}, {'question': '2. 3) พัฒนาเรียนรู้และการสอนของตนเองในการสอนของตนเอง'}, {'question': '2. 4) พัฒนาเรียนรู้และการสอนของตนเองในการสอนของตนเอง'}, {'question': '6. นโยบายสาธารณะใด ของประเทศในภาพยนตร์เชิงสารคดี เรื่อง "Where to invade next?" ที่เกี่ยวข้องกับประเทศไทย ที่ถ้านักศึกษาเป็นนายกรัฐมนตรี จะนำมาเป็นแนวทางในการบริหารประเทศให้กับประเทศไทย? และเพราะอะไร'}, {'question': '7. นี้สู้ตอบเพียง 1 นโยบาย วะบุประเทศมาด้วย!'}, {'question': '8. พร้อมกับ "โรงเรียนที่ดีที่สุดที่อุโรงเรียนที

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
for q in spark.streams.active:
    q.stop()